In [37]:
# -*- coding: utf-8 -*-

In [38]:
import os
import pandas as pd 
from glob import glob

import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter, Mecab,Kkma
from konlpy.utils import pprint

import chat_bot as cb
import Database_Handler as dh

In [39]:
ct = ctwitter()
ot = Twitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1384828, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


# 자연어 처리

## TextRank 

In [42]:
stopwords = open('./data/stopwordsList.txt','r',encoding='utf-8').readlines() + open('./data/newspress.txt','r',encoding = 'utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

참고
[인용사이트](http://excelsior-cjh.tistory.com/entry/TextRank%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EB%AC%B8%EC%84%9C%EC%9A%94%EC%95%BD)

# 분석

In [51]:
import networkx
import re
 
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            #print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both

    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

In [ ]:
path = './data/pre_data/keywords_by_mecab-naver*'
extracted_file = glob(path)
extracted = dict()
for file in extracted_file:
    ex = pickle.load(open(file,'rb'))
    extracted.update(ex)
outlen = str(len(extracted_file))
print (len(extracted))
outname = './data/pre_data/keywords_by_mecab-naver_'+outlen

In [64]:
extract_keywords_dict = dict()
site = 'Naver'
#site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
useCollection = dh.Use_Collection(useDb, collection)
keywordList = list()
collection = useCollection.find({'site':site})
count = 0
for data in collection:
    idIs = data['_id']._ObjectId__id
    tis2 = html.unescape(data['title'])+'. '+html.unescape(data['mainText'])
    if data['keywords'] !='NaN':
        keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
    else:
        pass
    if not idIs.hex() in extracted:
        print (count)
        tr = TextRank(window=10, coef=0.0)
        stopword = list(map(lambda x: mecab.pos(x), stopwords))
        stopword = list(itertools.chain.from_iterable(stopword))
        tr.load(RawTagger(tis2,mecab), lambda w: w not in stopword and (w[1] in ['VA','NNG','NNB','NNP']))
        tr.build()
        kw = tr.extract(0.125)
        if len(kw) != 0:
            extract_keywords_dict[idIs.hex()] = kw
        else:
            kw = tr.extract(0.25)
            if len(kw) != 0:
                extract_keywords_dict[idIs.hex()] = kw
            else:
                kw = tr.extract(0.375)
                extract_keywords_dict[idIs.hex()] = kw
        count +=1
pickle.dump(extract_keywords_dict, open(outname,'wb'))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194


KeyboardInterrupt: 

In [65]:
pickle.dump(extract_keywords_dict, open(outname,'wb'))